<a href="https://colab.research.google.com/github/kbrezinski/CS224W-GraphML/blob/main/notebooks/practice_deep_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print(torch.__version__)

from platform import python_version
print(python_version())

1.11.0+cu113
3.7.13


In [3]:
!pip install torch-scatter torch-sparse \
 torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-1.11.0+cu113.html -q
#!pip install ogb

In [4]:
import os
import numpy as np

from torch_geometric.data import Dataset, Data, Batch
from torch_geometric.loader import ClusterData, NeighborLoader, DataLoader

In [ ]:
# torch_geometric.data.batch.Batch 
# -creates batch of disconencted graphs from list

# torch_geometric.data.data.Data
# -creates single graph object

# torch_geometric.data.cluster.ClusterData/ClusterLoader
# -group nodes into smaller subgraphs and load them in batches for faster computation

# torch_geometric.data.sampler.NeighborSampler
# -samples specific number of nodes in neighbor
# -sample training nodes only using node_idx

In [5]:
import pickle

rows = np.random.choice(100, 500)
cols = np.random.choice(100, 500)

data = dict(x = torch.rand((100, 16), dtype=torch.float), # 100 nodes, 16 features)
            edge_index = torch.tensor([rows, cols]), # (2, 500) random edges
            edge_attr = np.random.choice(3, 500), # 500 edges, choose from 0, 1 or 2
            y = torch.rand(100).round().long(),
)  

with open('./data/g1.pickle', 'wb') as f:
  pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  import sys


In [12]:
g = Data(**data)
g

Data(x=[100, 16], edge_index=[2, 500], edge_attr=[500], y=[100])

In [13]:
# create batch of graphs using from_data_list
g2 = g
batch = Batch().from_data_list([g, g2])

In [6]:
# cluster data in 5 partitions, loop through N/5 nodes and retrieve edges
cluster = ClusterData(g, 5)
for c in cluster:
  print(c)
  break

NameError: ignored

In [ ]:
# for 4 nodes in the graph, sample (3 * 4), (10 * 40) neighbors for each iteration
sampler = NeighborLoader(g, num_neighbors=[3, 10],
                         batch_size=4, shuffle=False,
                         input_nodes=None) #data.train_mask,)
for s in sampler:
  print(s)
  break

Data(x=[49, 16], edge_index=[2, 65], edge_attr=[500], y=[49], batch_size=4)


In [16]:
import pickle
# pre-transform - 
# tranform - 
# pre-filter - 

# torch_geometric.transforms - list of functions transformations for graphs

class SampleDataset(Dataset):
  def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
    super().__init__(root, transform, pre_transform, pre_filter)

  @property
  def raw_file_names(self):
    return ['g1.pickle']
    
  @property
  def processed_file_names(self):
    return ['g1.pt']

  def download(self):   
    rows = np.random.choice(100, 500)
    cols = np.random.choice(100, 500)

    data = dict(x = torch.rand((100, 16), dtype=torch.float), # 100 nodes, 16 features)
                edge_index = torch.tensor([rows, cols]), # (2, 500) random edges
                edge_attr = np.random.choice(3, 500), # 500 edges, choose from 0, 1 or 2
                y = torch.rand(100).round().long(),
    )  

    with open('data/raw/g1.pickle', 'wb') as f:
      pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

  def process(self):

    # create data object
    for file_name in self.raw_file_names:
      with open(os.path.join('data/raw', file_name), 'rb') as f:
        data = pickle.load(f)

        graph = Data(**data)

        if self.pre_filter is not None:
          continue

        if self.pre_transform is not None:
          self.data = self.pre_transform(data)

      torch.save(self.data, os.path.join('data/processed',  file_name.split('.')[0] + '.pt'))

  def len(self):
    return len(self.processed_file_names)

  def get(self, idx):
    return torch.load(os.path.join('data/processed', f'g{idx}.pt'))
    

dataset = SampleDataset(root='data')

In [17]:
dataloader = DataLoader(dataset, batch_size=1)

In [18]:
for i in dataloader:
  print(i)
  break

FileNotFoundError: ignored

In [ ]:
# example transforms
# pre-transform - does so once its downloaded
# tranform - does so after its downloaded and retreived

transform = transforms.Compo se([
              transforms.AddTrainValTestMask('train_rest', num_val=500, num_test=500),
              transforms.TargetIndegree(),
])